# JSC370 Recommendation Systems Assignment

## Questions your report should answer:

### What value you could add with this data?
  - E.g. improve recommendations, suggest friends based on mutual interests, suggest new content to produce / acquire (e.g. Netflix)

### How to measure success?
  - E.g. Dollars of revenue, chance of liking a selected movie, increasing star-hours watched.  Must make proposed counterfactuals explicit - e.g. not making recommendation, or recommending worse movies.

### Look at the data.
  - Try to find outliers / problems in data.
  - Give a sense of the range of certainty or amount of data per decision that needs to be made.  What is the distribution of number of ratings?
  - Look for signs of saturation / overly-coarse granularity.  What is the distribution of ratings?  Could we expect to do better with finer-grained ratings?
  - What things depend on e.g. time?

### Brainstorm complementary sources of data.
  - E.g. More detailed information about movies, the movies themselves, professional ratings

### Brainstorm a comprehensive model.
  - What are all the factors that could conceivably influence someone's rating?  E.g. a family member uses their account, they want to seem artsy so rate movies highly that they don't actually want to watch, they misunderstand which movie is being rated, their tastes change over time, the movies available to rate change over time, etc.

### Propose a model staircase (series of more sophisticated models)
  
  - You don't have to implement every model you propose!
  - It doesn't have to be a strict staircase (each model doesn't have to incorporate the one before it).
  - You can propose models that you don't know how to implement.
  - Ideally, differences between results will make it clear which inputs / parts of the model are important.
  - You need to implement a different number of refinements depending on your group size.
      - 1 person: PMF and one variant.
      - 2 people: PMF and two variant.
      - 2 people: PMF and three variants.
  - Suggested variants:
      - Using user or movie side info instead of latents
      - Using user or movie side info in addition to latents.
      - Allowing user latents to be a linear function of time.
      - Making the prediction function non-linear.
      - Changing the training loss from squared error to something else (e.g. a Dirichlet distribution).

### Fit the models + do sanity checks
  - This is the coding part of the assignment.
  - Include one or two sanity checks, e.g. that accuracy is lower for users with fewer ratings, or that prize-winning movies that you've heard of have higher-than-average ratings.  

### Report evidence it works, expected value added, conditions for accurate use.
  - Spell out what question the model is answering.  E.g. "the person logged into this account, on this day, if they watched this movie..."
  - Spell out what question your train/test splits were answering.  E.g. predicting behavior of a random existing user vs a new user.  At a randomly chosen time, or a time after the training data was gathered?  Should correspond to how to shuffled + split the data when training.
  - Make improvement clear in terms of your metrics over some baseline.  E.g. "our best model has x% lower RMSE than simply guessing the mean".
  - Make a link between your improvement of this metric and some metric that a decision-maker might care about.  E.g. "Each batch of 10 suggestions is approx, y% more likely to contain a movie that a user will rate 5 stars.E.g. "If users give up after looking at 50 suggestions on average, using our most accurate system will lead to z% more movies watched and rated 5 stars per user per login".  It's OK to estimate numbers, just make it clear where you're doing so.
  - Make statements about what would increase or decrease the accuracy of the model over time.  E.g. many new users joining would hurt accuracy, long-time users sticking around will increase accuracy over time.

Assignments will be graded according to [these rubrics](https://jsc370.github.io/2020/assignment_rubrics.html)

### Imports

In [168]:
# import required libraries
!pip install wget
import os
import os.path

import matplotlib.pyplot as plt
import wget

import jax.numpy as jnp
import numpy.random as npr
from jax.api import jit, grad

import pandas as pd
import numpy as np

## Support functions and variables

In [ ]:
wget.download("https://github.com/MIE451-1513-2019/course-datasets/raw/master/ml-100k.zip")
!unzip ml-100k.zip
MOVIELENS_DIR = "ml-100k"

Archive:  ml-100k.zip
replace ml-100k/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls {MOVIELENS_DIR}

In [ ]:
def getData(folder_path, file_name):
    fields = ['userID', 'itemID', 'rating', 'timestamp']
    data = pd.read_csv(os.path.join(folder_path, file_name), sep='\t', names=fields)
    return data
rating_df = getData(MOVIELENS_DIR, 'u.data')

In [ ]:
rating_df.head()

In [ ]:
num_users = len(rating_df.userID.unique())
num_items = len(rating_df.itemID.unique())
print("Number of users:", num_users)
print("Number of items:", num_items)

In [ ]:
def dataFrameToArray(rating_df):
    """
        INPUT: 
            data: pandas DataFrame. columns=['userID', 'itemID', 'rating' ...]
            num_row: int. number of users
            num_col: int. number of items
            
        OUTPUT:
            matrix: 2D numpy array. 
    """
    matrix = np.zeros((rating_df.shape[0], 4), dtype=np.int32)
    # Populate the matrix based on the dataset
    for (index, userID, itemID, rating, timestamp) in rating_df.itertuples():
        matrix[index, :] = np.array([userID, itemID, rating, timestamp])
    return matrix

big_array = dataFrameToArray(rating_df)
plt.plot(big_array[:,0])

# Make train, validation, and test splits
train_mat = big_array[:80000, :]
valid_mat = big_array[80000:90000, :]
test_mat =  big_array[90000:100000, :]

train_inputs, train_targets = train_mat[:, :2], train_mat[:, 2].astype(jnp.float32)
test_inputs,  test_targets  = test_mat[:, :2],  test_mat[:, 2].astype(jnp.float32)


In [ ]:
# Model implementation in JAX (https://github.com/google/jax)

# Optimization hyperparameters
step_size = 100.0
train_iters = 1000

# Model hyperparameters
num_factors = 3
init_scale = 0.1
latent_param_regularization = 0.1

# Init parameters
user_latents  = np.random.randn(num_users, num_factors) * init_scale
movie_latents = np.random.randn(num_items, num_factors) * init_scale
params = (user_latents, movie_latents)

# Actual model
mean_rating = jnp.mean(train_targets)
def pmf_predict(user_latents, movie_latents, inputs):
  (user_index,   movie_index) = (inputs[:, 0], inputs[:, 1])
  return jnp.sum(user_latents[user_index] * movie_latents[movie_index], axis=1) + mean_rating

def regularization_loss(user_latents, movie_latents):
  return latent_param_regularization * (jnp.mean(user_latents**2) + jnp.mean(movie_latents**2))

def prediction_mse(params, inputs, targets):
  preds = pmf_predict(*params, inputs)
  return (preds - targets)**2

def training_loss(params, inputs, targets):
  return regularization_loss(*params) + jnp.mean(prediction_mse(params, inputs, targets), axis=0)

# One training step
@jit  # Pre-compiles the function to speed up training.
def sgd_update(params, i):  # Stochastic gradient descent
  (grads_user, grads_movie) = grad(training_loss)(params, train_inputs, train_targets)
  (user_latents, movie_latents) = params
  return (user_latents  - step_size * grads_user,  # one step of gradient descent
          movie_latents - step_size * grads_movie)

# Main training loop
for i in range(train_iters):
  params = sgd_update(params, i)
  if i % 100 == 0:
    # Print current progress
    train_loss = training_loss(params, train_inputs, train_targets)
    train_rmse = jnp.sqrt(prediction_mse(params, train_inputs, train_targets))
    test_rmse  = jnp.sqrt(prediction_mse(params, test_inputs, test_targets))
    print(i, train_loss, jnp.mean(train_rmse), jnp.mean(test_rmse))


In [ ]:
plt.plot(pmf_predict(*params, test_mat[0:1000, :2]), test_mat[0:1000, 2].astype(jnp.float32), '.')

# How well does predicting the mean do?
print(jnp.sqrt(jnp.mean((test_mat[:, 2] - mean_rating)**2)))


In [ ]:
(user_latents, movie_latents) = params
plt.plot(user_latents[:, 0], user_latents[:, 1], '.')


In [ ]:
plt.plot(movie_latents[:, 0], movie_latents[:, 1], '.')

In [ ]:
# Sort movies by latent factors, and report "meaning" of the different factors.
m_cols = ['movie_id', 'title', 'release_date']
movies = pd.read_csv(os.path.join(MOVIELENS_DIR, 'u.item'), sep='|', names=m_cols, usecols=range(3),
                     encoding='latin-1')

least_ixs  = np.argsort(movie_latents[:, 1])[:10]
most_ixs  = np.argsort(movie_latents[:, 1])[-10:]

print(movies.title[most_ixs])
print(movies.title[least_ixs])

# Suggestion: Filter out movies with few ratings, since their estimates will be noisiest.

# Suggestion: Try different numbers of latent factors and see how accuracy + meaning of factors changes.

# Suggestion: Include user side info as predictors and see how accuracy + meaning of factors changes.



The cell below is taken from the MIE1513 version of this course - it's just there to give you ideas for other metrics for evaluating your predictions.

In [ ]:
class CrossValidation(object):
    def __init__(self, metric, data_path=MOVIELENS_DIR):
        """
            INPUT:
                metric: string. from['RMSE','P@K','R@K']
        """
        self.folds = self._getData(MOVIELENS_DIR)
        self.metric_name = metric
        self.metric = self._getMetric(self.metric_name)
        
    def _getMetric(self, metric_name):
        """
            Don't change this
        """
        switcher = {
            'RMSE': self.rmse,
            'P@K': self.patk,
            'R@K': self.ratk,
            'RPrecision': self.rprecision
        }
        
        return switcher[metric_name]
    
    @staticmethod
    def rmse(data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        return sqrt(mean_squared_error(data[pred], data[true]))
    
    # Precision at k
    def patk(self, data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            k: top-k items retrived
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        prediction = self.getMatrix(data, num_users, num_items, pred)
        testSet =  self.getMatrix(data, num_users, num_items, true)
    
        # Initialize sum and count vars for average calculation
        sumPrecisions = 0
        countPrecisions = 0

        # Define function for converting 1-5 rating to 0/1 (like / don't like)
        vf = np.vectorize(lambda x: 1 if x >= 4 else 0)

        for userID in range(num_users):
            # Pick top K based on predicted rating
            userVector = prediction[userID,:]
            topK = nlargest(k, range(len(userVector)), userVector.take)

            # Convert test set ratings to like / don't like
            userTestVector = vf(testSet[userID,:]).nonzero()[0]

            # Calculate precision
            precision = float(len([item for item in topK if item in userTestVector]))/len(topK)

            # Update sum and count
            sumPrecisions += precision
            countPrecisions += 1

        # Return average P@k
        return float(sumPrecisions)/countPrecisions
    
    # Recall at k
    def ratk(self, data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            k: top-k items relevant
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        prediction = self.getMatrix(data, num_users, num_items, pred)
        testSet =  self.getMatrix(data, num_users, num_items, true)
        # Initialize sum and count vars for average calculation
        sumRecalls = 0
        countRecalls = 0

        # Define function for converting 1-5 rating to 0/1 (like / don't like)
        vf = np.vectorize(lambda x: 1 if x >= 4 else 0)

        for userID in range(num_users):
            # Pick top K based on predicted rating
            userVector = prediction[userID,:]
            topK = nlargest(k, range(len(userVector)), userVector.take)

            # Convert test set ratings to like / don't like
            userTestVector = vf(testSet[userID,:]).nonzero()[0]

            # Ignore user if has no ratings in the test set
            if (len(userTestVector) == 0):
                continue

            # Calculate recall
            recall = float(len([item for item in topK if item in userTestVector]))/len(userTestVector)

            # Update sum and count
            sumRecalls += recall
            countRecalls += 1

        # Return average R@k
        return float(sumRecalls)/countRecalls

    def rprecision(self, data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame.
            k: top-k items relevant
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        prediction = self.getMatrix(data, num_users, num_items, pred)
        testSet    = self.getMatrix(data, num_users, num_items, true)
        # Initialize sum and count vars for average calculation
        sumRPs = 0
        countRPs = 0

        # Define function for converting 1-5 rating to 0/1 (like / don't like)
        vf = np.vectorize(lambda x: 1 if x >= 4 else 0)

        for userID in range(num_users):
            # Pick top K based on predicted rating
            userVector = prediction[userID, :]


            # Convert test set ratings to like / don't like
            userTestVector = vf(testSet[userID, :]).nonzero()[0]

            # Ignore user if has no ratings in the test set
            if (len(userTestVector) == 0):
                continue

            topK = nlargest(len(userTestVector), range(len(userVector)), userVector.take)
            # Calculate recall
            rp = float(len([item for item in topK if item in userTestVector])) / len(userTestVector)

            # Update sum and count
            sumRPs += rp
            countRPs += 1

        # Return average R@k
        return float(sumRPs) / countRPs

    @staticmethod
    def getMatrix(rating_df, num_users, num_items, column_name):
        matrix = np.zeros((num_users, num_items))
    
        for (index, userID, itemID, value) in rating_df[['userID','itemID', column_name]].itertuples():
            matrix[userID-1, itemID-1] = value
            
        return matrix
    
    @staticmethod
    def _getData(data_path):
        """
            Don't change this function
        """
        folds = []
        data_types = ['u{0}.base','u{0}.test']
        for i in range(1,6):
            train_set = getData(data_path, data_types[0].format(i))
            test_set = getData(data_path, data_types[1].format(i))
            folds.append([train_set, test_set])
        return folds
    
    def run(self, algorithms, num_users, num_items, k=1):
        """
            5-fold cross-validation
            algorithms: list. a list of algorithms. 
                        eg: [user_cosine_recsys, item_euclidean_recsys]
        """
        
        scores = {}
        for algorithm in algorithms:
            print('Processing algorithm {0}'.format(algorithm.getPredColName()))
            fold_scores = []
            for fold in self.folds:
                algorithm.reset()
                algorithm.predict_all(fold[0], num_users, num_items)
                prediction = algorithm.evaluate_test(fold[1])
                pred_col = algorithm.getPredColName()
                fold_scores.append(self.metric(prediction, k, num_users, num_items, pred_col))
                
            mean = np.mean(fold_scores)
            ci_low, ci_high = stats.t.interval(0.95, len(fold_scores)-1, loc=mean, scale=stats.sem(fold_scores))
            scores[algorithm.getPredColName()] = [fold_scores, mean, ci_low, ci_high]
            
        results = scores    
    
        return results
            